In [53]:
# Source https://stuvel.eu/python-rsa-doc/index.html
# Install rsa
# pip install rsa

#step 1 , create public and private keys , source https://stuvel.eu/python-rsa-doc/usage.html#generating-keys
import rsa
pubkey, privkey = rsa.newkeys(512) # refer Keysize (bits)
print(f"Public Key(share with all):{pubkey} \n , private key(status staus secret: {privkey})")

Public Key(share with all):PublicKey(8710559449331836352688480383277906697605729099539121020080527260411100988948443161579603143900055391140682867744988573617976178699164383672333810544197367, 65537) 
 , private key(status staus secret: PrivateKey(8710559449331836352688480383277906697605729099539121020080527260411100988948443161579603143900055391140682867744988573617976178699164383672333810544197367, 65537, 2107695679501720568242884506736973685256140074469252195499290496904027333903056016429804950551640347441212143813705100950486617420745265118953322361115937, 6352618835024915386129972160535910436656071834879588059862584566457288148007382063, 1371176151999944912389693589550337397018622609572903285489680301988167609))


In [54]:
# save keys to files, refer https://stuvel.eu/python-rsa-doc/reference.html#functions
# PEM is fileformat - more reading at https://stackoverflow.com/questions/22743415/what-are-the-differences-between-pem-cer-and-der
with open("public.PEM","wb") as fh:
    fh.write(pubkey.save_pkcs1("PEM"))

with open("private.PEM","wb") as fh:
    fh.write(privkey.save_pkcs1("PEM"))


<img src="images/public_key_file.png" width=400 height=100 />

<img src="images/privaate_key_file.png" width=400 height=100 />

In [55]:
# load those keys, source - https://stuvel.eu/python-rsa-doc/usage.html
private_key,public_key='',''
with open('private.PEM', mode='rb') as fh:
    private_key=rsa.PrivateKey.load_pkcs1(fh.read()) 

with open('public.PEM', mode='rb') as fh:
    public_key=rsa.PublicKey.load_pkcs1(fh.read()) 

print(f"read keys\n {private_key} \n {public_key}") # private keys are for my eyes only

print(f"created vs read:{privkey == private_key}, {pubkey == public_key} ") # they matched, so all good so far

read keys
 PrivateKey(8710559449331836352688480383277906697605729099539121020080527260411100988948443161579603143900055391140682867744988573617976178699164383672333810544197367, 65537, 2107695679501720568242884506736973685256140074469252195499290496904027333903056016429804950551640347441212143813705100950486617420745265118953322361115937, 6352618835024915386129972160535910436656071834879588059862584566457288148007382063, 1371176151999944912389693589550337397018622609572903285489680301988167609) 
 PublicKey(8710559449331836352688480383277906697605729099539121020080527260411100988948443161579603143900055391140682867744988573617976178699164383672333810544197367, 65537)
created vs read:True, True 


In [56]:
secure_this_message="I am API Key"
coded_message=rsa.encrypt(secure_this_message.encode("utf8"),public_key) # rsa.encrypt need bytes, so string.encode("utf8") convert to bytes
print(f"Coded: {coded_message}")
print(f"Decoded:{rsa.decrypt(coded_message,private_key).decode('utf8')}")

Coded: b'Cs\r\x83\x03\x856ZF\x19\xafAe\xc9\xc9\xd5\x82{\xabj\xfa\xfc\x06\x82\x83\xd2d\xe6i-/\xd9\x9e\xcb\n\xe1\xf9f\x86\x8eLYL\x0b\x1f\x9cj4+\xad\xd4\xaf2\x96\x81\xc5"\xa3\xe4\x9bE\xfa\xbb\xaa'
Decoded:I am API Key


In [ ]:
#Let's encrypy the OpenAI key and save to ini file
import configparser
sec_parser=configparser.ConfigParser()
sec_parser.read('secret.ini')
openai_api_key=sec_parser['OpenAI']['openai_api_key'] # convert it to secure, had a file, so reading and converting, use case be simple as read and convert

config = configparser.ConfigParser()
config.read('private_git.ini')
config['OpenAI']={}

config['OpenAI']['openai_api_key_encoded']=rsa.encrypt(openai_api_key.encode("utf8"),public_key).decode("windows-1251")
with open('private_git.ini', 'w') as configfile:
    config.write(configfile)

'''
way to check the encoding!
source - my SO post - https://stackoverflow.com/questions/75253022/error-while-parsing-json-file-probally-a-hidden-value-on-the-json-content/75253495#75253495
import chardet
file_encoding=chardet.detect(en_key)['encoding'] # detect the encoding
print(f"file(bytes) encoding:{file_encoding}") # print encoding
'''

In [83]:
# read from the config file
config = configparser.ConfigParser()
config.read('private_git.ini')
openai_api_key_encoded=config['OpenAI']['openai_api_key_encoded'].encode("windows-1251")


In [87]:
from langchain.llms import OpenAI
import os

secured_api_key = config['OpenAI']['openai_api_key_encoded'].encode("windows-1251") # set open AI key as environment variable
secured_api_key=rsa.decrypt(secured_api_key,private_key).decode('windows-1251')
llm_model_openai=OpenAI(openai_api_key=secured_api_key)
llm_model_openai("Who won the recent soccer world cup?") # does not know about 2022 , as never seen this data

'\n\nThe most recent FIFA World Cup was held in 2018 and was won by France.'

## OpenAI access, env variable using following code: <br>
api_key = os.environ.get("OPENAI_API_KEY") refer ln#42     openai-python/openai/__init__.py <br> 
so instead of OpenAI() , we can initialize the object as OpenAI(openai_api_key=secured_api_key) i.e passing the parameter value. 